In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input
from keras import applications
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16

from sklearn.svm import LinearSVC
from sklearn.metrics import average_precision_score, precision_recall_curve, classification_report
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import os
import numpy as np
from collections import Counter
import pandas as pd


In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes, cmap=plt.cm.Blues):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Normalize
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           ylabel='True label',
           xlabel='Predicted label')
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    
    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    plt.show(ax)

In [ ]:
def classification(x_train, y_train, x_test, y_test):
    # define the model's parameters
    svc = svm.SVC(gamma='scale', decision_function_shape='ovo')
    
    # train the model on the train set
    svc.fit(x_train, y_train.ravel())
    
    # predict test set labels
    y_pred = svc.predict(x_test)
    
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    print("Precision:",metrics.precision_score(y_test, y_pred, average='micro'))
    print("Recall:",metrics.recall_score(y_test, y_pred, average='micro'))
    print(metrics.classification_report(y_test, y_pred))
    
    # Plot normalized confusion matrix
    plot_confusion_matrix(y_test, y_pred, classes=['Airplane', 'Bird', 'Dog', 'Horse'])

In [ ]:
def knn_classification(x_train, y_train, x_test, y_test):
    # create KNN Classifier
    knn = KNeighborsClassifier(n_neighbors=5)

    # train the model using the training sets
    knn.fit(x_train, y_train)
    
    # predict test set labels
    y_pred = knn.predict(x_test)

    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("Precision:",metrics.precision_score(y_test, y_pred, average='micro'))
    print("Recall:",metrics.recall_score(y_test, y_pred, average='micro'))
    print(metrics.classification_report(y_test, y_pred))
    
    # Plot normalized confusion matrix
    plot_confusion_matrix(y_test, y_pred, classes=['Airplane', 'Bird', 'Dog', 'Horse'])

In [ ]:
def prepare_data():
    labels = {0:'Airplane', 1:'Automobile', 2:'Bird', 3:'Cat', 4:'Deer', 5:'Dog', 6:'Frog', 7:'Horse', 8:'Ship', 9:'Truck'}

    # The data, split between train and test sets:
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # preprocess
    x_test_new = []
    y_test_new = []
    x_train_new = []
    y_train_new = []
    
    # Array of counter, each class has one, it was used to test different number of samples
    count = [0, 0, 0, 0]
    for i, j in zip(x_train, y_train):
        if (j==0):
            if(count[0]<5000):
                x_train_new.append(i)
                y_train_new.append(j)
                count[0]+=1

        elif (j==5):
            if(count[1]<5000):
                x_train_new.append(i)
                y_train_new.append(j)
                count[1]+=1

        elif (j==7):
            if(count[2]<5000):
                x_train_new.append(i)
                y_train_new.append(j)
                count[2]+=1

        elif (j==2):
            if(count[3]<5000):
                x_train_new.append(i)
                y_train_new.append(j)
                count[3]+=1

    x_train = np.array(x_train_new).astype('float32')
    y_train = np.array(y_train_new)

    # Repeat for the test set
    count = [0, 0, 0, 0]
    for i, j in zip(x_test, y_test):
        if (j==0):
            if(count[0]<500):
                x_test_new.append(i)
                y_test_new.append(j)
                count[0]+=1

        elif (j==5):
            if(count[1]<500):
                x_test_new.append(i)
                y_test_new.append(j)
                count[1]+=1

        elif (j==7):
            if(count[2]<500):
                x_test_new.append(i)
                y_test_new.append(j)
                count[2]+=1

        elif (j==2):
            if(count[3]<500):
                x_test_new.append(i)
                y_test_new.append(j)
                count[3]+=1

    x_test = np.array(x_test_new).astype('float32')
    y_test = np.array(y_test_new)

    # Normalize numbers
    x_test = x_test / 255.
    x_train = x_train / 255.

    print("train shape:",x_train.shape)
    print("train labels shape:",y_train.shape)
    print("test shape:",x_test.shape)
    print("test labels shape:",y_test.shape)
    
    return x_train, y_train, x_test, y_test

In [ ]:
def tsne_plot(x1, y1):
    print('Building t-SNE...')
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1)
    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y1 == 0), 0], X_t[np.where(y1 == 0), 1], marker='o', color='g', 	linewidth='1', alpha=0.8, label='Airplane')
    plt.scatter(X_t[np.where(y1 == 2), 0], X_t[np.where(y1 == 2), 1], marker='o', color='r', linewidth='1', alpha=0.8, label='Bird')
    plt.scatter(X_t[np.where(y1 == 5), 0], X_t[np.where(y1 == 5), 1], marker='o', color='b', linewidth='1', alpha=0.8, label='Dog')
    plt.scatter(X_t[np.where(y1 == 7), 0], X_t[np.where(y1 == 7), 1], marker='o', color='black', linewidth='1', alpha=0.8, label='Horse')
    
    plt.legend(loc='best')
    plt.show()

# Basic analysis

In [ ]:
# load the data
x_train, y_train, x_test, y_test = prepare_data()

# show the first n images in train and test set
n=6
plt.figure(figsize=(18, 4))
for i in range(n):
    # display train
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_train[i].reshape(32, 32, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    #display test
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(x_test[i].reshape(32, 32, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

# First CUT

In this first attempt we just cut at the end of the convolutional blocks

In [ ]:
# load the data
x_train, y_train, x_test, y_test = prepare_data()

# build the VGG16 network
vgg16 = VGG16(include_top=False, weights='imagenet', input_shape = (32,32,3))
#print(vgg16.summary())

# change the model type
model = Sequential()
for layer in vgg16.layers:
    model.add(layer)

# add the flatten output
model.add(Flatten())

#print(model.summary())

In [ ]:
# obtain features
train_features = model.predict(x_train)
print(train_features.shape)
test_features = model.predict(x_test)

In [ ]:
# run SVM
classification(train_features, y_train, test_features, y_test)

# Second CUT

In this second attempt we are going to cut just before the last block of convolutions

In [ ]:
# load the data
x_train, y_train, x_test, y_test = prepare_data()

# build the VGG16 network
base_model = VGG16(include_top=False, weights='imagenet', input_shape = (32,32,3))
vgg16 = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

# change the model type
model = Sequential()
for layer in vgg16.layers:
    model.add(layer)

# add the flatten output
model.add(Flatten())

In [ ]:
# obtain features
train_features = model.predict(x_train)
print(train_features.shape)
test_features = model.predict(x_test)

In [ ]:
# run SVM
classification(train_features, y_train, test_features, y_test)

# Third CUT

In the last attempt we are going to cut after the third block of convolutions, which means one block before the Second CUT's version.

In [ ]:
# load the data
x_train, y_train, x_test, y_test = prepare_data()

# build the VGG16 network
base_model = VGG16(include_top=False, weights='imagenet', input_shape = (32,32,3))
vgg16 = Model(inputs=base_model.input, outputs=base_model.get_layer('block3_pool').output)

# change the model type
model = Sequential()
for layer in vgg16.layers:
    model.add(layer)

# add the flatten output
model.add(Flatten())

In [ ]:
# obtain features
train_features = model.predict(x_train)
print(train_features.shape)
test_features = model.predict(x_test)

In [ ]:
# run SVM
classification(train_features, y_train, test_features, y_test)

## Commento veloce su K-NN vs SVM
k-nn produce risultati meno performanti rispetto a svm, in particolare è influenzato molto dalla scelta dei vicini: al crescere del numero i risultati diminuiscono e ha il suo ottimo con 1 solo vicino, tuttavia non è sufficiente e SVM è migliore sia con LinearSVM sia con SVC one-versus-one.